# Converting System to Handle Polyhedra

In [1]:
using Test
using JuMP
using Gurobi
using LinearAlgebra
using Polyhedra

In [18]:
include("../../lib/measurement-scheduling0.jl")
include("../../tests/mpc_functions_tests.jl")
include("../../tests/linear_system_test.jl")
include("../../tests/alap_estimation_schedule_test.jl")

Test Summary:                | Pass  Total
compute_Sx0() Function Tests |    1      1
Test Summary:               | Pass  Total
compute_Sw() Function Tests |    4      4
Test Summary:                | Pass  Total
compute_C_M() Function Tests |    1      1
Test Summary:             | Pass  Total
LinearSystem Class Struct |    1      1
Test Summary: | Pass  Total
check() tests |    3      3
Test Summary: | Pass  Total
x_dim() Tests |    3      3
Test Summary: | Pass  Total
y_dim() Tests |    4      4
Test Summary:                       | Pass  Total
define_simple_eta_HPolytope() Tests |    2      2
find_est_error_bound_from_time_0_to_T Tests: Error During Test at /Users/kwesirutledge/Documents/Michigan/Research/measurement-scheduling0/tests/linear_system_test.jl:143
  Got exception outside of a @test
  MethodError: no method matching LinearSystem(::Float64, ::Int64, ::Int64, ::MixedMatHRep{Float64,Array{Float64,2}}, ::MixedMatHRep{Float64,Array{Float64,2}})
  Closest candidates are:
    

LoadError: LoadError: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.
in expression starting at /Users/kwesirutledge/Documents/Michigan/Research/measurement-scheduling0/tests/linear_system_test.jl:143

In [3]:
# function wrapper for counting evaluations
# continuous_lipm, discretized_lipm = get_lipm()
# println("Continuous System")
# println(continuous_lipm)
# println("Discretized System")
# println(discretized_lipm)

num_drones = 2
drone_sys = get_drones(num_drones)

# Get Drone Systems
println(" "); println("Coucou")
println("Starting Drone work.")
get_drones(1)

 
Coucou
Starting Drone work.


LinearSystem([1.0 0.0 0.1 0.0; 0.0 1.0 0.0 0.1; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0], [0.0 0.0; 0.0 0.0; 0.1 0.0; 0.0 0.1], [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0], [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0], [0.0, 0.0], HalfSpace([1.0, 0.0, 0.0, 0.0], 0.5) ∩ HalfSpace([0.0, 1.0, 0.0, 0.0], 0.5) ∩ HalfSpace([0.0, 0.0, 1.0, 0.0], 0.5) ∩ HalfSpace([0.0, 0.0, 0.0, 1.0], 0.5) ∩ HalfSpace([-1.0, 0.0, 0.0, 0.0], 0.5) ∩ HalfSpace([0.0, -1.0, 0.0, 0.0], 0.5) ∩ HalfSpace([0.0, 0.0, -1.0, 0.0], 0.5) ∩ HalfSpace([0.0, 0.0, 0.0, -1.0], 0.5), HalfSpace([1.0, 0.0], 0.2) ∩ HalfSpace([0.0, 1.0], 0.2) ∩ HalfSpace([-1.0, 0.0], 0.2) ∩ HalfSpace([0.0, -1.0], 0.2), HalfSpace([1.0, 0.0], 0.2) ∩ HalfSpace([0.0, 1.0], 0.2) ∩ HalfSpace([-1.0, 0.0], 0.2) ∩ HalfSpace([0.0, -1.0], 0.2))

In [4]:
sigma_m = [1 zeros((1,1))]

O_3 = packet_loss_observability_mat( drone_sys , sigma_m )
println("O_3=")
println(O_3)

println(typeof(O_3))
println(typeof(zeros((size(O_3,1),1))))

H_x0 = [
        Matrix(1.0I, num_drones*4, num_drones*4) ; 
        Matrix(-1.0I, num_drones*4, num_drones*4) ; 
        O_3 ;
        -O_3
        ]
h_x0 = [ 
        1.0 * ones((num_drones*4*2,));
        0.0*zeros((size(O_3,1)*2,))
        ]
X0 = polyhedron(hrep(H_x0,h_x0))

H_zT = [
        Matrix(1.0I, num_drones*2, num_drones*2) ; 
        Matrix(-1.0I, num_drones*2, num_drones*2)
        ]

h_zT = [  6.5 ; 1.5 ; 6.5 ; 1.5 ; 1.5 ; 1.5 ; 1.5; 1.5  ]

ZT = polyhedron(hrep(H_zT,h_zT))

J = compute_Sx0( drone_sys.A , length(sigma_m))
T = length(sigma_m)
Zt_ol = [ zeros((num_drones*2,x_dim(drone_sys)*T)) I(num_drones*2) zeros((num_drones*2,x_dim(drone_sys)-num_drones*2)) ]*J*X0

println(Zt_ol)

println(test_set_containment(Zt_ol,ZT))

println(fulldim(X0))

N_O3 = nullspace(O_3) 
println("N_O3 = ")
println(N_O3)

O_3=
[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
Array{Float64,2}
Array{Float64,2}
convexhull([0.2, 0.2, 1.0, 1.0], [-0.2, 0.2, -1.0, 1.0], [0.2, -0.2, 1.0, -1.0], [-0.2, -0.2, -1.0, -1.0], [0.2, 0.2, 1.0, 1.0], [-0.2, 0.2, -1.0, 1.0], [0.2, -0.2, 1.0, -1.0], [-0.2, -0.2, -1.0, -1.0], [0.2, 0.2, 1.0, 1.0], [-0.2, 0.2, -1.0, 1.0], [0.2, -0.2, 1.0, -1.0], [-0.2, -0.2, -1.0, -1.0], [0.2, 0.2, 1.0, 1.0], [-0.2, 0.2, -1.0, 1.0], [0.2, -0.2, 1.0, -1.0], [-0.2, -0.2, -1.0, -1.0])
Academic license - for non-commercial use only - expires 2021-05-07
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
termination_status = INFEASIBLE
INFEASIBLE
8
N_O3 = 
[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0;

In [5]:
tempV = polyhedron(drone_sys.V)

test_set_containment(tempV,tempV)

Academic license - for non-commercial use only - expires 2021-05-07
termination_status = OPTIMAL


OPTIMAL::TerminationStatusCode = 1

In [6]:
tempV2 = vrep(tempV)
num_vertices = size(tempV2.V,1)
for vertex_index = 1:size(tempV2.V,1)
    println(vertex_index, string(rand(Float64,1)))
    println()
end

theta = rand(Float64,num_vertices)
theta = theta/sum(theta)

sample = transpose(tempV2.V)*theta
println("sample = ")
println(sample)

println("sample_polytope(tempV)")
println(sample_polytope(tempV))

1[0.9955491739730222]

2[0.8132689722303093]

3[0.7337155503653705]

4[0.3922344527664612]

5[0.15161012479648317]

6[0.267974642295038]

7[0.14545804601356505]

8[0.17222080278595464]

9[0.9855813209615545]

10[0.6530593954423887]

11[0.22919403007772]

12[0.8098352244982039]

13[0.5420776029312444]

14[0.567875880413202]

15[0.17941587164241835]

16[0.5480303382786342]

sample = 
[-0.013341306355429405, 0.0285413991389062, -0.030300011645184886, 0.0027902073568410214]
sample_polytope(tempV)
[-0.00566880129812819, -0.0755876762284258, 0.031708801552992605, 0.012332363695373267]
